# Flow-Shop Problem-Solving with Google `FunSearch`

> Authors: CUI Guangyuan, XU Zhuojun, LI Songyan

This notebook is the main entrance of our work on the flow-shop problems' solving using Google `FunSearch` via various attempts. Mainly, the applications of existing methods on the problem-solving can be divided into three categories: baseline experiments including the applications of Google `FunSearch` and `OR-Tools` as well as the existing heuristics. New approaches are also proposed in this research, including:

- Trials on different kinds of prompts (Prompt Engineering)
- Using priority-based method
- FunSearch with Curriculum Learning

## Dataset

## Baseline Experiments

### Existing Heuristics

### Google `OR-Tools`

Import necessary libraries:

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import numpy as np

from ortools.sat.python import cp_model

Define the function to read cases from instances in desired formats:

In [ ]:
def read_cases(path):
    cases = []
    with open(path, 'r') as f:
        alldata = f.readlines()
        first_line = alldata[0].split()
        n_jobs, n_machines = int(first_line[0]), int(first_line[1])

        for i in range(1, len(alldata)):
            line = alldata[i]
            jobs_cases = []
            data = line.split()
            for d in range(0, len(data), 2):
                jobs_cases.append((int(data[d]), int(data[d+1])))
            cases.append(jobs_cases)

    return (n_jobs, n_machines), cases

Define the function for plotting Gantt chart of the result:

In [ ]:
def plot_gantt_chart(result_job_schedule, num_jobs, num_machines,
                     title="Flow-Shop Gantt Chart"):
    fig, ax = plt.subplots(figsize=(25, 12))

    colors = list(mcolors.TABLEAU_COLORS.values())
    if num_jobs > len(colors):
        random.seed(4487)
        colors = []
        for _ in range(num_jobs):
            colors.append(f'#{random.randint(0, 0xFFFFFF):06x}')

    for job_id, machine_id, stime, etime in result_job_schedule:
        duration = etime - stime
        rect = patches.Rectangle(
            (stime, num_machines - machine_id - 1),
            duration,
            0.8,
            linewidth=1,
            edgecolor='black',
            facecolor=colors[job_id],
            alpha=0.6,
            label=f'Job-{job_id}' if machine_id == 0 else ''
        )
        ax.add_patch(rect)

        rx, ry = rect.get_xy()
        ax.text(
            rx + duration / 2,
            ry + 0.4,
            f'J{job_id}',
            ha='center',
            va='center',
            color='black',
            fontweight='light'
        )

    ax.set_xlim(0, max([endtime for _, _, _, endtime in result_job_schedule]) + 1)
    ax.set_ylim(0, num_machines)

    ax.set_yticks(np.arange(num_machines) + 0.4)
    ax.set_yticklabels([f'Machine {num_machines - i - 1}' for i in range(num_machines)])

    ax.grid(True, axis='x', linestyle='--', alpha=0.5)

    handles = [patches.Patch(color=colors[i], label=f'Job {i}') for i in range(num_jobs)]
    ax.legend(handles=handles, loc='upper right', ncol=min(5, num_jobs))

    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel('Machines')

    plt.tight_layout()
    plt.show()

Define the main process of problem solving:

In [ ]:
def solve_flowshop(num_jobs, num_machines, jobs_data):
    model = cp_model.CpModel()


    """Create interval variables"""
    intervals = {}
    for job_id in range(num_jobs):
        for task_id, (machine_id, duration) in enumerate(jobs_data[job_id]):
            # a job is consisted of multiple tasks

            unique_id = f'{job_id}-{machine_id}-{task_id}'
            start = model.NewIntVar(0, 10000, f's-{unique_id}')
            end = model.NewIntVar(0, 10000, f'e-{unique_id}')
            interval = model.NewIntervalVar(start, duration, end, f'interval-{unique_id}')

            # uniquely identified by job ID and machine ID
            # since a task can only be executed on a machine
            intervals[(job_id, machine_id)] = (start, end, interval)


    """Add constraints on the order"""
    for job_id in range(num_jobs):
        for task_id in range(1, num_machines):
            # (task number = machine number) in flow-shop

            prev_task_machine = jobs_data[job_id][task_id - 1][0]
            cur_task_machine = jobs_data[job_id][task_id][0]

            # start time of current task must be larger than the end time of previous task
            model.Add(intervals[(job_id, cur_task_machine)][0]
                      >= intervals[(job_id, prev_task_machine)][1])


    """Add constraints on the machine conflicts"""
    for machine_id in range(num_machines):
        machine_intervals = [
            intervals[(job_id, machine_id)][2]
            for job_id in range(num_jobs)
        ]
        model.AddNoOverlap(machine_intervals)


    """Setting Objects"""
    case_object = model.NewIntVar(0, 10000, 'makespan')
    model.AddMaxEquality(case_object, [
        intervals[(job_id, num_machines-1)][1]
        for job_id in range(num_jobs)
    ])
    model.Minimize(case_object)


    """Solving"""
    solver = cp_model.CpSolver()
    stat = solver.Solve(model)


    """Results"""
    result_schedule = []
    if stat == cp_model.OPTIMAL:
        # exists optimal solution
        print(f'Optimal Makespan: {solver.ObjectiveValue()}')
        for job_id in range(num_jobs):
            for machine_id in range(num_machines):
                start = solver.Value(intervals[(job_id, machine_id)][0])
                duration = jobs_data[job_id][machine_id][1]
                end = start + duration
                print(f'Task-{machine_id} of Job-{job_id} is scheduled on Machine-{machine_id}: {start} ~ {end}')
                result_schedule.append((job_id, machine_id, start, end))
    else:
        # No optimal solution
        print('No solution found')

    return result_schedule


In [ ]:
case_set_name = 'reeves'
case_no = 20
case_path = f'../data/{case_set_name}/{case_set_name}{case_no}.txt'

(n_jobs, n_machines), jobs_data = read_cases(case_path)
result_schedule = solve_flowshop(n_jobs, n_machines, jobs_data)

plot_gantt_chart(result_schedule, n_jobs, n_machines, f'Result of case: {case_set_name}-{case_no}')

### Naïve Application on FunSearch

## New Approaches

### Prompt Engineering

### Priority-based Method

### FunSearch with Curriculum Learning

This is our main key improvements that has made on the base FunSearch Framework. The basic idea for the new framework is applying Curriculum Learning in the evolving process of FunSearch.

Specifically, a single iteration of evolving are turned into multiple iterations. In this framework, we call them 'Stages':
- The input instances are divided into various stages, starting from 'easy' instances all the way up to 'complicated' instances
    - Degrees of complication are defined manually
- At each stage, FunSearch is executed only with the instances belong to that stage, and gets the result
- If the result of current stage is higher than the baseline score, then it enters the next stage, namely a more complicated stage
    - Baseline function that provides baseline score is given by the raw function at each stage before the actual evolving
- If the result of current stage is lower than the baseline score, it keeps trying until it reaches the maximum number of attempts defined in advance, or gets a better score and escape current stage
- The final output of this framework can either be the 'half-evolved' or 'completely-evolved' function due to the maximum attempts limit

> See directory `implementation_cl` for the detailed implementation of this framework.


## Evaluations